In [63]:
import xml.etree.ElementTree as ET
from datetime import datetime,timezone, timedelta
import pandas as pd
import numpy as np
import re
import glob

In [65]:
ns = {'': "http://www.w3.org/2005/Atom"}
days = ['Monday', 'Tuesday', 'Wednesday', 'Thursday', 'Friday', 'Saturday', 'Sunday']

for filename in glob.glob('*.xml'):
    tree=ET.parse(filename)
    root = tree.getroot()

    updated_date_string=root.find('updated',ns).text # eg, 2022-04-15T09:05:49Z
    forecast_datetime = datetime.strptime(updated_date_string, '%Y-%m-%dT%H:%M:%SZ').replace(tzinfo=timezone.utc)
    day0=days[ forecast_datetime.weekday() ]

    forecast_day_ahead_count=0
    prediction_records=[]
    for entry in root.findall('entry',ns):
        if entry.find('category',ns).get('term') == 'Weather Forecasts':
            # Title contains the brief forecast text. Summary has more detail.
            #print('\t' + entry.find('summary',ns).text)

            title=entry.find('title',ns).text
            title_split=title.split()

            # Figure out the date of this prediction using weekday name
            day1=title_split[0].split(':')[0]
            if day1 != day0:
                day0=day1
                forecast_day_ahead_count=forecast_day_ahead_count+1

            entry_date=forecast_datetime+timedelta(days=forecast_day_ahead_count)

            # AM/PM
            prediction_period='AM'
            if title_split[1]=='night:':
                prediction_period='PM'
            prediction_text=title.split(':')[1]

            # POP
            pop=0
            pop_match=re.search('POP ([0-9]+)\%',prediction_text)
            if pop_match:
                pop=pop_match.group(1)
            elif 'rain' in prediction_text or 'snow' in prediction_text:
                pop=100

            # Temp
            pop_match=re.search('(High|Low)( minus| plus)? ([0-9]+)',prediction_text) # that ' minus' (note whitepace)
            high_low=''
            temp=np.nan
            if pop_match:
                high_low = pop_match.group(1)
                temp=int(pop_match.group(3))
                if pop_match.group(2) == ' minus': # keep this whitespace
                    temp=temp*(-1)

            prediction_records.append({ 'as_of_date_utc': forecast_datetime, 
                               'prediction_interval_days': forecast_day_ahead_count,
                               'prediction_period': prediction_period,
                               'pop': pop,
                               'high_low': high_low,
                               'temp': temp,
                               'raw_prediction': prediction_text})

            forecast_count=forecast_count+1

        #print(entry.find('category',ns).get('term'))
    
forecasts=pd.DataFrame(prediction_records)